In [1]:
import requests
from bs4 import BeautifulSoup
import re
import tqdm
import json

In [ ]:
# INSERT HERE YOUR COOKIES
caelum_login_token = ''
jsessionid = ''
alura_userid = ''

In [2]:
# categories (schools) to find courses
categories = ['programacao', 'front-end', 'data-science', 'devops', 'design-ux', 'mobile', 'inovacao-gestao']


base_link = 'https://cursos.alura.com.br'


# Headers to requests
headers_complete = {
    'authority': 'cursos.alura.com.br',
    'cache-control': 'max-age=0',
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="99", "Google Chrome";v="99"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Linux"',
    'upgrade-insecure-requests': '1',
    'dnt': '1',
    'content-type': 'application/x-www-form-urlencoded',
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-user': '?1',
    'sec-fetch-dest': 'document',
    'referer': 'https://cursos.alura.com.br',
    'accept-language': 'en-US,en;q=0.9,pt-BR;q=0.8,pt;q=0.7',
    'cookie': f'alura.userId={alura_userid}; darkmode=false; theatermode=false; caelum.login.token={caelum_login_token}; JSESSIONID={jsessionid}',
}

headers = {
    'authority': 'cursos.alura.com.br',
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="99", "Google Chrome";v="99"',
    'dnt': '1',
    'sec-ch-ua-mobile': '?0',
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.74 Safari/537.36',
    'content-type': 'application/json; charset=UTF-8',
    'accept': 'application/json, text/javascript, */*; q=0.01',
    'x-requested-with': 'XMLHttpRequest',
    'sec-ch-ua-platform': '"Linux"',
    'origin': 'https://cursos.alura.com.br',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-mode': 'cors',
    'sec-fetch-dest': 'empty',
    'referer': 'https://cursos.alura.com.br',
    'accept-language': 'en-US,en;q=0.9,pt-BR;q=0.8,pt;q=0.7',
    'cookie': f'caelum.login.token={caelum_login_token}; alura.userId={alura_userid}; theatermode=false; darkmode=false; JSESSIONID={jsessionid}',
}

In [3]:
# for each category request their html document
category_soups = [BeautifulSoup(requests.get(f'{base_link}/category/{category}',
                                            headers=headers).text,
                                'html.parser') for category in categories]

# for each categorie crawl for courses
courses_link = list(set([tag['href'] for soup in category_soups for tag in soup.find_all("a", class_="course-card__course-link")]))

In [ ]:
# for each course found
for course in tqdm.tqdm(courses_link[:]):
    try:
        # set referer to specify the "current" page
        headers['referer'] = f'{base_link}{course}'
        
        # try to enroll course if It's not started yet
        response = requests.get(f'{base_link}{course.replace("course", "courses")}/tryToEnroll', headers=headers)
        
        # jsession_id = response.headers['Set-Cookie'].split(';')[0]
        # headers['cookie'] = headers['cookie'].rsplit('JSESSIONID')[0] + jsession_id
        
        # get the next pending task
        response = requests.get(f'{base_link}{course}/continue', headers=headers)
        task = response.url.split('.br')[-1]
        
        # while there are pending tasks 
        while True:
            
            # if the link is a certificate = Course completed
            if task.split('/')[1] == 'certificate':
                break
            
            # if get course completed land
            if task.split('/')[-2] == 'completeCourse':
                response = requests.get(f'{base_link}{task}', headers=headers_complete)
                headers['referer'] = f'{base_link}{task}'
                task_land = BeautifulSoup(response.text, 'html.parser')
                data = {
                'kind': 'END_COURSE_EVALUATION',
                'nextTaskId': '',
                'netPromoteScore': '10',
                'testimony': 'muito bom'
                }
                a = task_land.find('form', class_='evaluation-form')
                response = requests.post(f'{base_link}{a["action"]}', headers=headers_complete, data=data)
                _ = requests.get(f'{base_link}/recommendations/course/{a["action"].split("/")[-2]}', headers=headers_complete)
                _ = requests.get(f'{base_link}{course}', headers=headers_complete)
                break
            
            # block to answer the questions
            response = requests.get(f'{base_link}{task}', headers=headers)
            headers['referer'] = f'{base_link}{task}'
            task_land = BeautifulSoup(response.text, 'html.parser')
            
            if 'congratulation' in task:
                task = task_land.find('a', class_='finishSection-content-button')['href']
                break
            
            task_type = task_land.find('li', class_='task-menu-nav-item--selected').find('use')['xlink:href'].split('#')[1]
            if task_type in ['VIDEO', 'STUDYING_TOGETHER']:
                data = {
                    'courseCode': course.split('/')[-1],
                    'videoTaskId': task.split('/')[-1]
                }
                _ = requests.post(f'{base_link}{task}/mark-video', headers=headers, data=data)
            elif task_type in ['TEXT_CONTENT', 'COMPLEMENTARY_INFORMATION', 'DO_AFTER_ME', 'WHAT_WE_LEARNED', 'HQ_EXPLANATION',
                            'SETUP_EXPLANATION', 'VARIATION', 'CHALLENGE', '-1']:
                pass
            elif task_type in ['SINGLE_CHOICE', 'PRACTICE_CLASS_CONTENT']:
                alternative_id = task_land.find('li', {"data-correct" : "true"})['data-alternative-id']
                data = json.dumps({'taskId': task.split('/')[-1],
                        'alternatives': [alternative_id]})
                section_number = task_land.find('section')['data-section-number']
                _ = requests.post(f'{base_link}{course}/section/{section_number}/singlechoice/answer', headers=headers, data=data) 
            elif task_type ==  'MULTIPLE_CHOICE':
                data = json.dumps({'taskId': task.split('/')[-1],
                        'alternatives': [x['data-alternative-id'] for x in task_land.find_all('li', {"data-correct" : "true"})]})
                section_number = task_land.find('section')['data-section-number']
                _ = requests.post(f'{base_link}{course}/section/{section_number}/multiplechoice/answer', headers=headers, data=data)
            elif task_type == 'OPEN_QUESTION':
                data = '{"taskId":%s,"alternatives":[],"answeredText":"a","answeredTextHighlight":"<p>a</p>\\n"}' % task.split('/')[-1]
                section_number = task_land.find('section')['data-section-number']
                _ = requests.post(f'{base_link}{course}/section/{section_number}/openquestion/answer', headers=headers, data=data)
            else:
                # if unknown type
                print('unknown tasktype: ' + task_type)
                print(task)
                raise Exception
            
            
            
            # get the next task
            task = task_land.find('a', {
                'class': 'task-actions-button task-body-actions-button task-actions-button-next bootcamp-next-button bootcamp-primary-button-theme'
            })
            task = task['href']
            headers['referer'] = f'{base_link}{task}'
            task = requests.get(f'{base_link}{task}', headers=headers).url.split('.br')[-1]
            if 'curso-online-' in task:
                task = '/course/' + task.split('/curso-online-')[-1]
            if '#' in task:
                task_land = BeautifulSoup(requests.get(f'{base_link}{task}', headers=headers).text, 'html.parser')
                task = task_land.find('a', {'class': 'courseSectionList-section'}).find('a')['href']

    except:
        # if any error: print course to debug
        print(course)